In [2]:
import pymongo
from splinter import Browser
from bs4 import BeautifulSoup
from datetime import datetime
import os
import platform

In [3]:

def init_browser():
    if platform.system().lower() == 'windows'.lower():
        executable_path = {
            'executable_path': 
            os.path.join(os.getcwd(), 'chromedriver.exe')}
        return Browser('chrome', **executable_path, headless=False)
    else:
        return Browser('chrome')
    
def get_html(browser, url = "https://marvelcinematicuniverse.fandom.com/wiki/Category:Movies?from="):
    browser.visit(url)
    html = browser.html
    return html
def get_listings(html):
    
    soup = BeautifulSoup(html, "html.parser")
    listings = soup.find_all('div', class_='category-page__members')
    return listings
    
def scrape(browser, url):
   
    html = get_html(browser, url)
    listings = get_listings(html)
    return listings
def access_db(dbname, collectionname):
    conn = 'mongodb://localhost:27017'
    client = pymongo.MongoClient(conn)
    db = client[dbname]
    collection = db[collectionname]
    return db, collection
    
def scrape_many(search_terms, collection, browser, base_url):
    
    for search_term in search_terms:
        url = base_url + search_term
        titles=scrape(browser, url)
        collection.insert_many([{'searchterm':search_term,'title':title.text,
                                 'datetime':datetime.now(),'order':order} 
                                    for order, title in enumerate(titles)])
def main():
    db, collection = access_db('mcu','searchrank')
    browser = init_browser()
    base_url = "https://marvelcinematicuniverse.fandom.com/wiki/Category:Movies?from="
    search_terms = ['Phase Two Movies']
    scrape_many(search_terms, collection, browser, base_url)
    return
main()

In [4]:
db, collection = access_db('mcu','searchrank')

In [7]:
[data['title'] for data in collection.find()]

['Spider-Man',
 '\n\t\t\t\t\t\t\t\thttps://marvelcinematicuniverse.fandom.com/wiki/Spider-Man\t\t\t\t\t\t\t',
 'Spider-Man: Far From Home',
 '\n\t\t\t\t\t\t\t\thttps://marvelcinematicuniverse.fandom.com/wiki/Spider-Man:_Far_From_Home\t\t\t\t\t\t\t',
 'Spider-Man 3',
 '\n\t\t\t\t\t\t\t\thttps://marvelcinematicuniverse.fandom.com/wiki/Spider-Man_3\t\t\t\t\t\t\t',
 'Spider-Man Suit',
 '\n\t\t\t\t\t\t\t\thttps://marvelcinematicuniverse.fandom.com/wiki/Spider-Man_Suit\t\t\t\t\t\t\t',
 'Spider-Man: Homecoming',
 '\n\t\t\t\t\t\t\t\thttps://marvelcinematicuniverse.fandom.com/wiki/Spider-Man:_Homecoming\t\t\t\t\t\t\t',
 'Spider-Man: Far From Home/Gallery',
 '\n\t\t\t\t\t\t\t\thttps://marvelcinematicuniverse.fandom.com/wiki/Spider-Man:_Far_From_Home/Gallery\t\t\t\t\t\t\t',
 'Spider-Man/Quote',
 '\n\t\t\t\t\t\t\t\thttps://marvelcinematicuniverse.fandom.com/wiki/Spider-Man/Quote\t\t\t\t\t\t\t',
 'Spider-Man/Gallery',
 '\n\t\t\t\t\t\t\t\thttps://marvelcinematicuniverse.fandom.com/wiki/Spider-Man/Ga